# Decoding a neural network from a JSON file to Python, then encoding the hessian from laplace.py to a CSV (multi-class classification)

- Step 1: Read the dataset
- Step 2: Initialize model
- Step 3: Read the weights and biases from the JSON file, then load them into the previously initialized model
- Step 4: Generate the hessians and write them to CSV files

In [16]:
# imports
from laplace import Laplace

import numpy as np
import pandas as pd
import torch

import json

import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

torch.manual_seed(43)
torch.set_printoptions(sci_mode=False)

### Step 1: Read the dataset

In [17]:
# Import data from csv

# Load data from CSV file using pandas
df = pd.read_csv('data_multi.csv')

# Split the dataframe into x and y tensors
x = torch.from_numpy(df[['x1', 'x2']].to_numpy()).to(torch.float32)
y = torch.from_numpy(df['y'].to_numpy(dtype=int))

X = x.T

y_unique = torch.unique(y)
y_indices = y - 1
y_indices = y_indices.long()
y_train = nn.functional.one_hot(y_indices, num_classes=len(y_unique)).float()
y_train

tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1.,

### Step 2: Initialize model

In [18]:
# Init model

data = list(zip(x, y_train))
n_hidden = 3
D = X.shape[0]  # == 2
out_dim = y_train.shape[1]  # == 4
model = nn.Sequential(
    nn.Linear(D, n_hidden),
    nn.Sigmoid(),
    nn.Linear(n_hidden, out_dim)
)

loss_fn = nn.CrossEntropyLoss()

### Step 3: Read the weights and biases from the JSON file, then load them into the previously initialized model

In [19]:
with open('nn_multi.json') as fin:
    nn_json_str = fin.read()
    nn_json = json.loads(nn_json_str)

nn_json

[{'bias': [2.2643619, 1.4519827, -0.82282156],
  'weight': [[4.5985866, -0.33530173, -0.3286117],
   [0.020469662, 1.8550323, -1.9706149]]},
 {'bias': [-2.3268585, 0.75011677, -2.5603814, 0.76309216],
  'weight': [[6.1197248, -8.305814, 6.425951, -8.373602],
   [-0.5828485, -6.455521, -9.15029, 3.3358324],
   [-9.106588, 3.1897228, -0.39506513, -6.322449]]}]

In [20]:
assert len(model.state_dict()) == 2 * len(nn_json)
iter_states = iter(model.state_dict())

# for layer in model.state_dict():
#     print(layer)
for layer_read in nn_json:
    state_w = next(iter_states)
    state_b = next(iter_states)
    tensor_w = torch.tensor(layer_read['weight']).T
    tensor_b = torch.tensor(layer_read['bias']).T
    model.state_dict()[state_w].data.copy_(tensor_w)
    model.state_dict()[state_b].data.copy_(tensor_b)

In [21]:
for val in model.state_dict().values():
    display(val)

tensor([[ 4.5986,  0.0205],
        [-0.3353,  1.8550],
        [-0.3286, -1.9706]])

tensor([ 2.2644,  1.4520, -0.8228])

tensor([[ 6.1197, -0.5828, -9.1066],
        [-8.3058, -6.4555,  3.1897],
        [ 6.4260, -9.1503, -0.3951],
        [-8.3736,  3.3358, -6.3224]])

tensor([-2.3269,  0.7501, -2.5604,  0.7631])

In [22]:
y_hat = torch.argmax(torch.softmax(model.forward(x), dim=1), dim=1) + 1

y_hat == y

tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, Tr

In [23]:
print(model(x))

tensor([[  3.1009, -13.7871,  -5.0352,  -4.4544],
        [  3.1989, -13.9695,  -5.2335,  -4.3036],
        [  3.2100, -14.0110,  -5.2845,  -4.2748],
        [  2.9709, -13.5555,  -4.7860,  -4.6491],
        [  3.2099, -14.0098,  -5.2828,  -4.2755],
        [  3.2044, -13.9704,  -5.2317,  -4.3004],
        [  3.2034, -13.9785,  -5.2434,  -4.2963],
        [  3.2090, -13.8829,  -5.1094,  -4.3472],
        [  3.2101, -14.0105,  -5.2837,  -4.2751],
        [  3.2091, -14.0099,  -5.2853,  -4.2737],
        [  3.1903, -13.8480,  -5.0714,  -4.3762],
        [  3.1848, -13.8804,  -5.1186,  -4.3607],
        [  3.2097, -14.0104,  -5.2839,  -4.2751],
        [  3.2099, -14.0109,  -5.2847,  -4.2746],
        [  3.2099, -14.0104,  -5.2837,  -4.2752],
        [  3.2100, -14.0108,  -5.2842,  -4.2749],
        [  3.2058, -14.0051,  -5.2866,  -4.2714],
        [  3.2106, -13.9959,  -5.2634,  -4.2830],
        [  3.2269, -13.1274,  -4.0649,  -4.7627],
        [  3.2079, -14.0082,  -5.2861,  -4.2725],


### Step 4: Generate the hessians and write them to CSV files

NOTE: In this case of regression, it is no longer necessary to convert the format of the hessian to match the one from PyTorch. This is because both the input and output layers have size == 1. Therefore all weights and biases are vectors, and for a vector x: vec(x) == vec(transpose(x)).

In [24]:
la = Laplace(model, 'classification',
             subset_of_weights='all',
             hessian_structure='full')

la.fit(DataLoader(TensorDataset(x, y_train), batch_size=1))

hessian = la.H
torch.set_printoptions(precision=4, sci_mode=True)
array = hessian.numpy()
np.savetxt('hessian_multi_all_full_ggn.csv', array, delimiter=',')

In [25]:
la = Laplace(model, 'classification',
             subset_of_weights='last_layer',
             hessian_structure='full')

la.fit(DataLoader(TensorDataset(x, y_train), batch_size=1))

hessian = la.H
array = hessian.numpy()
np.savetxt('hessian_multi_ll_full_ggn.csv', array, delimiter=',')

In [26]:
la = Laplace(model, 'classification',
             subset_of_weights='subnetwork',
             subnetwork_indices = torch.LongTensor([0, 5, 6, 8, 9, 20, 21, 24]),
             hessian_structure='full')

la.fit(DataLoader(TensorDataset(x, y_train), batch_size=1))

hessian = la.H
array = hessian.numpy()
np.savetxt('hessian_multi_subnet_full_ggn.csv', array, delimiter=',')

In [27]:
from laplace.curvature import AsdlEF

In [28]:
la = Laplace(model, 'classification',
             subset_of_weights='all',
             hessian_structure='full', backend=AsdlEF)

la.fit(DataLoader(TensorDataset(x, y_train), batch_size=1))

hessian = la.H
torch.set_printoptions(precision=4, sci_mode=True)
array = hessian.numpy()
np.savetxt('hessian_multi_all_full_empfisher.csv', array, delimiter=',')

In [29]:
la = Laplace(model, 'classification',
             subset_of_weights='last_layer',
             hessian_structure='full', backend=AsdlEF)

la.fit(DataLoader(TensorDataset(x, y_train), batch_size=1))

hessian = la.H
array = hessian.numpy()
np.savetxt('hessian_multi_ll_full_empfisher.csv', array, delimiter=',')

In [30]:
la = Laplace(model, 'classification',
             subset_of_weights='subnetwork',
             subnetwork_indices = torch.LongTensor([0, 5, 6, 8, 9, 20, 21, 24]),
             hessian_structure='full', backend=AsdlEF)

la.fit(DataLoader(TensorDataset(x, y_train), batch_size=1))

hessian = la.H
array = hessian.numpy()
np.savetxt('hessian_multi_subnet_full_empfisher.csv', array, delimiter=',')

In [33]:
la = Laplace(model, 'classification',
             subset_of_weights='all',
             hessian_structure='kron', backend=AsdlEF)

la.fit(DataLoader(TensorDataset(x, y_train), batch_size=1))

hessian = la.H
torch.set_printoptions(precision=4, sci_mode=True)
array = hessian.numpy()
np.savetxt('hessian_multi_all_kron_empfisher.csv', array, delimiter=',')

NotImplementedError: Unavailable through Backpack.

In [ ]:
la = Laplace(model, 'classification',
             subset_of_weights='last_layer',
             hessian_structure='kron', backend=AsdlEF)

la.fit(DataLoader(TensorDataset(x, y_train), batch_size=1))

hessian = la.H
torch.set_printoptions(precision=4, sci_mode=True)
array = hessian.numpy()
np.savetxt('hessian_multi_ll_kron_empfisher.csv', array, delimiter=',')